# Answering Business Questions Using SQL

In this guided project we will practice using more advanced SQL queries to answer business questions pertaining to our chinook database.



In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Explore all the database tables

In [2]:

%%sql
SELECT * from sqlite_master

Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


### Most Popular Genres in the USA

Lets find out which genres sell the most tracks in the USA

In [3]:
%%sql
WITH usa_sales AS
    (SELECT * 
    FROM invoice
    INNER JOIN invoice_line ON invoice_line.invoice_id= invoice.invoice_id 
    WHERE billing_country = 'USA'
    )

SELECT genre.name, COUNT(genre.name) genre_count,
        CAST(COUNT(genre.name)as float)/ (SELECT COUNT(*)
        FROM usa_sales) * 100  percent
FROM track
LEFT JOIN usa_sales ON track.track_id = usa_sales.track_id  
LEFT JOIN genre ON genre.genre_id = track.genre_id
WHERE usa_sales.billing_country = 'USA'
GROUP BY genre.name
ORDER BY genre_count DESC


Done.


name,genre_count,percent
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


The top 3 genres are "Rock", "Alternative & Punk" and "Metal", with the "Rock" genre being the most succesful in terms of sales. 

### Employee Sales Performance

In the following cell we will be analyzing the top sales perfomers in the company. 

In [4]:
%%sql

WITH employee_sales AS
    (SELECT *
    FROM employee
    LEFT JOIN customer ON customer.support_rep_id = employee.employee_id )
    
    
SELECT first_name || ' ' || last_name employee_name, 
    SUM(invoice.total) total_sales,
    country
FROM employee_sales
LEFT JOIN invoice ON invoice.customer_id = employee_sales.customer_id
WHERE title LIKE '%sale%'
GROUP BY first_name
ORDER BY total_sales DESC

Done.


employee_name,total_sales,country
Jane Peacock,1731.510000000004,Canada
Margaret Park,1584.0000000000032,Canada
Steve Johnson,1393.9200000000028,Canada
Nancy Edwards,None,Canada


The top sales person is Jane Peacock from Canada. 

### Purchasing Power by Country

In thi section we will query the total number of sales, customers, the averyge values of sales per customer and the average order value

Here we join the largest database(invoice_line) with the customer and invoice databases using an INNER JOIN to access information about total sale value, average sale value per customer and average order value.

The values from "Other" are sorted to the bottom of the returned table

In [333]:
%%sql 


WITH customer_country AS 
(SELECT *,
CASE 
WHEN (SELECT COUNT(*)
        FROM customer
        WHERE customer.country = c.country
     ) = 1 THEN 'Other'
    ELSE c.country
    END AS country_count
FROM invoice_line 
INNER JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
INNER JOIN customer c ON c.customer_id = invoice.customer_id
)


SELECT *
FROM 
(SELECT 
 country_count , 
        COUNT(DISTINCT customer_id) total_customers,
        SUM(unit_price) total_sale_value,
        SUM(unit_price)/ COUNT(DISTINCT customer_id) average_sales_per_customer,
        SUM(unit_price)/ COUNT(DISTINCT invoice_id) average_order_value,
 CASE 
     WHEN country_count = 'Other' THEN 1
     ELSE 0
     END AS sort
FROM customer_country
GROUP BY country_count
ORDER BY sort ASC, total_sale_value DESC
)




Done.


country_count,total_customers,total_sale_value,average_sales_per_customer,average_order_value,sort
USA,13,1040.490000000008,80.03769230769292,7.942671755725252,0
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309,0
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647,0
France,5,389.0700000000021,77.81400000000042,7.781400000000042,0
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186,0
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034,0
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457,0
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835,0
India,2,183.1500000000002,91.5750000000001,8.72142857142858,0
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486,1
